In [1]:
import sys
import importlib
import os
# make the module available from the src directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
data_dir = "../datasets/artificial_data/3d_shapes"

def create_list():
    pairs = []
    all_files = os.listdir(data_dir)
    for filename in all_files:
        label = int(filename.split("_").pop().replace(".pt", ""))
        label_dict = { "size": label }
        file_path = os.path.join(data_dir, filename)
        pairs.append({"image": file_path, "label": label_dict})
    return pairs

pairs = create_list()
print(pairs)

[{'image': '../datasets/artificial_data/3d_shapes\\volume_0_label_17.pt', 'label': {'size': 17}}, {'image': '../datasets/artificial_data/3d_shapes\\volume_1000_label_17.pt', 'label': {'size': 17}}, {'image': '../datasets/artificial_data/3d_shapes\\volume_1001_label_16.pt', 'label': {'size': 16}}, {'image': '../datasets/artificial_data/3d_shapes\\volume_1002_label_17.pt', 'label': {'size': 17}}, {'image': '../datasets/artificial_data/3d_shapes\\volume_1003_label_20.pt', 'label': {'size': 20}}, {'image': '../datasets/artificial_data/3d_shapes\\volume_1004_label_18.pt', 'label': {'size': 18}}, {'image': '../datasets/artificial_data/3d_shapes\\volume_1005_label_15.pt', 'label': {'size': 15}}, {'image': '../datasets/artificial_data/3d_shapes\\volume_1006_label_18.pt', 'label': {'size': 18}}, {'image': '../datasets/artificial_data/3d_shapes\\volume_1007_label_23.pt', 'label': {'size': 23}}, {'image': '../datasets/artificial_data/3d_shapes\\volume_1008_label_27.pt', 'label': {'size': 27}}, {'

In [3]:
from src.transformers.TensorLoaderTransfomer import TensorLoader
from monai.transforms import Compose, LoadImaged, EnsureChannelFirstD, ScaleIntensityd, ToTensord, ResizeD, Lambdad, NormalizeIntensityD

from src.transformers.SliceTransformer import SliceAggregateTransform
importlib.reload(sys.modules['src.transformers.SliceTransformer'])

importlib.reload(sys.modules['src.transformers.TensorLoaderTransfomer'])
transforms = [
    TensorLoader(keys=["image"]),
    ResizeD(keys=["image"], spatial_size=(64, 64, 64), mode="area"),
    ScaleIntensityd(keys=["image"], minv=0.0, maxv=1.0, factor=1.0),
    # NormalizeIntensityD(keys=["image"], nonzero=True, channel_wise=True),
    ]

In [4]:
from src.dataLoaders.NiftiDataLoader2 import NiftiDataLoader
importlib.reload(sys.modules['src.dataLoaders.NiftiDataLoader2'])

<module 'src.dataLoaders.NiftiDataLoader2' from 'c:\\Datalogi\\Bachelor\\XAI-in-Medical-Image-Analysis\\src\\dataLoaders\\NiftiDataLoader2.py'>

In [5]:
dataset = 'niftiShort'
data_path = '../datasets/nifti/'
data_dir = f'{data_path}{dataset}'

# data_dir = 'C:/Users/askel/Downloads/NIFTY/NIFTY/'
# max = 8876
dim = 384
data_loader = NiftiDataLoader(data_dir=data_dir, 
                              batch_size=4,
                              spatial_resize=(dim, dim, dim),
                              cache_rate=0.2,
                              replace_rate=0.2,
                              transforms=transforms,
                              data_list=pairs,
                              )
# data_loader.load_data(visit_no=visit, subset_size=2058, cache="standard")
# data_loader.load_data(subset_size=84, cache="standard")z
#data_loader.load_data(subset_size=4400, cache="standard")
# data_loader.load_data(subset_size=8875, cache="standard")

data_loader.load_data(cache="smart")

File c:\Datalogi\Bachelor\XAI-in-Medical-Image-Analysis\src\dataLoaders\saved_data_lists\data_list.pkl does not exist.
Data list loaded: False
Using custom sampler: False
Subset size: 5000
Subset data length: 3999
Cache num: 799


Loading dataset: 100%|██████████| 500/500 [00:09<00:00, 51.86it/s]

batch_size: 4


In [7]:
from src.models.medical_models.monai_resnet import MonaiMedicalResNet
importlib.reload(sys.modules['src.models.medical_models.monai_resnet'])
importlib.reload(sys.modules['src.models.medical_models.base_medical'])

<module 'src.models.medical_models.base_medical' from 'c:\\Datalogi\\Bachelor\\XAI-in-Medical-Image-Analysis\\src\\models\\medical_models\\base_medical.py'>

In [8]:
fusion_model = MonaiMedicalResNet(
    num_epochs=50,
    learning_rate=0.001,
    weight_decay=1e-4,
    data_loader=data_loader,
    #dropout_rate=0.1,
    depth=18,
    pretrained=False
)

fusion_model.train()
fusion_model.evaluate()

Data loader train loader:  200 799
Image spatial dimensions:  torch.Size([4, 1, 64, 64, 64])
Number of input channels:  1
Spatial dims:  3
GPU:  cuda:0
Number of training images: 799
Epoch 1/50, Train Loss: 35.49139552406967, Val Loss: 31.05626920890808, R^2 Score: 0.07793164253234863, P-value: 2.1063706334700782e-10
Model saved at c:\Datalogi\Bachelor\XAI-in-Medical-Image-Analysis\src\models\medical_models\saved_models\MonaiMedicalResNet_18_799_height_torch.Size([4, 1, 64, 64, 64])_epoch_1_val_31.06_r2_0.08.pth
Epoch 2/50, Train Loss: 14.48882649242878, Val Loss: 21.73059525489807, R^2 Score: 0.3548133373260498, P-value: 1.1102230246251565e-16
Model saved at c:\Datalogi\Bachelor\XAI-in-Medical-Image-Analysis\src\models\medical_models\saved_models\MonaiMedicalResNet_18_799_height_torch.Size([4, 1, 64, 64, 64])_epoch_2_val_21.73_r2_0.35.pth
Epoch 3/50, Train Loss: 12.251900510936975, Val Loss: 10.123912369728089, R^2 Score: 0.6994185447692871, P-value: 1.1102230246251565e-16
Model saved

KeyboardInterrupt: 

In [ ]:
from src.XAI.VanillaSaliencyMedica3D import VanillaSaliency3D